# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
print(categories)

[{'id': 1, 'name': 'basketball'}]


## Helper code

In [10]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [11]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
#TEST_IMAGE_PATHS.extend([os.path.join(PATH_TO_TEST_IMAGES_DIR, 'dog_{}.jpeg'.format(i)) for i in range(1, 2)])
TEST_IMAGE_PATHS.extend([os.path.join(PATH_TO_TEST_IMAGES_DIR, 'dog_{}.jpeg'.format(i)) for i in range(1, 2)])

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [12]:
TEST_IMAGE_PATHS

['test_images\\image1.jpg',
 'test_images\\image2.jpg',
 'test_images\\dog_1.jpeg']

In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
img = cv2.imread(os.path.join(PATH_TO_TEST_IMAGES_DIR, 'pug1.jpg'), cv2.IMREAD_GRAYSCALE)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
plt.imshow(img, cmap='gray', interpolation='bicubic')
plt.xticks([]), plt.yticks([])
plt.plot([200, 300, 400], [200, 300, 300], 'c', linewidth=5)
plt.show()

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', gray)
    #cv2.imshow('frameColor', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.line(img, (0, 0), (200, 300), (255, 255, 255), 50)
cv2.rectangle(img, (500, 250), (1000, 500), (0, 0, 255), 15)
cv2.circle(img, (447, 63), 63, (0, 255, 0), -1)
pts = np.array([[100, 50], [200, 300], [700, 200], [500, 100]], np.int32)
pts = pts.reshape((-1, 1, 2))
cv2.polylines(img, [pts], True, (0, 255, 255), 3)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, 'OpenCV Tuts!', (10, 500), font, 6, (200, 255, 155), 13, cv2.LINE_AA)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        while True:
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
            cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

In [ ]:
# Images were found online by scraping the html for img tags and their src property. Easy with yahoo.
PATH_TO_TRAIN_IMAGES_DIR = 'basketball_images'
images = [
"https://tse3.mm.bing.net/th?id=OIP.ySsyta7lwgCVhgywGNdR9wFZC0&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP._ICtXB5vjMFn9kbN6CbIxQEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.IydvLPf3gaMzBgJQLOPrkgEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.TrJ1pIVfUv4nnDUdDvUl8QEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.Lg4dqBN7C4dc9BS4nJOluAEsDa&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.I9yh7AZZrCEAPXXFJ_c2eQEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.BXrqeW9tCAI6eVIiNfnJXgEsDG&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.MCmQdmjsEGQBjMO-2LCOZwEsDW&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.VUBnyiXRlhe4j4Og2xGDjwEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP._65-PBhjvLvILP7rqIVa6ADIEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.YKQxSdgyUza9-HC3BZMtKAEsDx&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.vHF8NAQnP3IQ5ZmFBJMn6gEsCy&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.9HIuOs7gsm1UIcG4ILdHbAEsCW&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.rgT18ovHFnGnsT2ggprOlwEsDX&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.VTB7peoKMan4GLckcMyPgwEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.tdnKc4__G3bEktkGPeC1OQEsDK&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.3liD8SWEQjOv2iO0uabE7QEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.2-CBhfskEsAVi4nSZQYHlwEsB9&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.ttaHMaO6N5QXOAo3Xajp-QEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.Z6OtFs4GnISOdd4jk2cDfwEsDa&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.LaJaXm61sgtbzakW_-fk3AEsDQ&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.Wf5wLahgSD3IggzVh4BatADHEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.tfKBQkfIlSKfKoSNNL7F2wEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.GwpBB_PjtjfvgdNX_D_zSQEsC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.bw9YcJkURC7B-XhB6NMIpgEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.xO8pjpXZn3PR3XaSEfF3xQEsDk&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.BwuKQU2dE-hxsB6FOQfy0AEsDF&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.D9ii-ajlWLLKB8kF_DTVJwEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.xwrGi1pYUinKuKh6yrEKBwEpDR&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.lXH8ZQtZbstfI1iJ8ZAg3gEyDL&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP._7R6ZLkxUUbOtYGkU0wAEAEsCz&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.5zFjWcHP1qziS9xzOrMxRAEsCz&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.8i1RAaxHmjSI3MDTba2dkQDbEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.v9oCkfl4L5-n1kezKndN_AEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.RpuLW_CEy2Xt7OyUPib8hAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.GW_W9JWxm0HfMVPCGSaV9AEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.elqOfnKiEMnR_EsslsfyWQEsCz&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.0Cc8bqeBTcH3YF41SEwOJAEsDg&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.O2SVy2PRT8oS16JJRmVZ8AEsCz&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.MaEKEIZgJ1hb-xPZkNO_ogEsDA&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.XFHlD5z8odiWwyBuSEXi9wEsC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.LpytSlL-ehBZFHjkxx_AewFNC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.yay3yrSbMbV6eflds6Dj2wEyDL&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.f-AD7LJpUHsWl5TkbQtFPgEsDd&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.E9d5G4jKsOLr994RrcxXoAEsDg&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.jNDFxKgbkSuS8FspJnnnGQD9Es&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.oDf_uozNwEL2mkZBCkwg-wEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.DcHx35KvJ7dPAvGtuNGQDwEqDR&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.CocF394JTRqb9Aav_yF6CAEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.ocNWISig1TwacSSeXZr2FAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.4XHi9XyDm3EEb2DYXvIuPAEsDc&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.U7yUoDbgBiEZsxZYjYPDZAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.uBkfvYMbfhscUrKrLv-W8wEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.V7TO51v05D8w6iF6zVq5yQEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.W0u_C0k_rEtHBTCcQ60lewEXDf&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.S1itt1W9IaajeW5AKeexfwE5DH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.gda7kyHs7sEgNdgtNdknfAEsEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.vyzXKU9bRsmagL46IaWZGAEsDx&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.7Z1572ue677JaF6d5sGtxAEsDV&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.jfVPdvxCHGfyIgNZVeI2BgDQEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.VIeSqtCdJ-NiR0an4Ec86QEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.LoiD8d8YkAtddivYjLr47wEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.C3UcXH2ehJZI8SMurJC0hADhEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.8i4g53nx5NQPmkuSDvJyXAEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.G5IDCZMbqR3uHR3Mb9vlOgEsDS&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.w2P7NV-nVj_oGL8O_aj_JQEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.q5k5m5X4erjPNdI6p8uibwEfEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.9HVzJURy2iR-q_a58gRnwwEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.9laCPmSev-_6_3YVLjzQawEuDO&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.50UBowMpeXjn9H_xTBJbVgEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.MWMBAcsnJjUA9h3l7407OwDcEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.0AdfjzRr-FOnq4LtI0Yk-wDhEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.3NV3i8UPqGJj5NouJk4UawDIDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.dY0v5ByBH0YKlycDwE9D6gEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP._zOnUUGejgCIfUrRLn3AwQEsC0&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.3umPY5EYfD-nDWKuV6NjxAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.7T3w2pAKdkvM-TvaeIYBhwEsCk&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.Gt3dWmHpGgWzN57pBG_zJgEsEe&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.RwMS4mclSXcxAJvn1PnqtwEsDU&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.ZXGFslYtE07q8yEo1LOb2AEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.TXCgF1jgcC-b_MMdtT2vpAEsBo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.Itzh3tJMZpNdVX9q1VVwmQEsDS&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.-oFVpcHMKeblGy__p1KeIAEsEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.H4GtY6W9R1efPDl734l_TQEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.ne42Wv4Od-iGOd-N25VcLQC7Es&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.34NO_9aHOb1-E2g0b3dF1gEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.17prONJyJeo46Y58hFOPDgEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.i9MYW6v0_Q46Cvm4KBIyJAEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.oKCG7wJKe_SQCoDYSsZSywEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.dspjFNbsxFEMECLbMSM3ZgEsCA&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.SY222zxx4ebG_oXZ_Ixj6wEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.MZW64XjY8lGHl4PcxBJrDwEsDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.V7XYrZ4ezsBCQ8w_lJaIegEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.6V1hQMTKfJNgcXm93HnYrwEsC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.IuwuQ-zrv75WNvlqxz67bgEsCW&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.htbAESXGRJEb7AbLupBudAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.4WnmOou1FZdsLDJZB6TtKgEsDA&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP._shRuGsaCgC4ZrG-t2kV6wEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.uOGT3Y-1pmjL5DTXyHV4lgEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.pr1DOxC1zwC3lOVEttrHnwEJEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.naFhm0j3n-Rt87LzHwLqBwEyDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.chgTPRZmlPXq7O41vCZpwQDIEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.9Z-arDhJrNjsp_dLFvvooQEyDL&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.FnQoXghBHAnZiDLLIQcAcwEhDX&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.0rHZ-r0lwbzWgWBCCrUlmgEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.6V1hQMTKfJNgcXm93HnYrwEsC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.bpVo0duL_qxCnVos83JcZwEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.FnQoXghBHAnZiDLLIQcAcwEhDX&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.sxZeDV6VaopQq-lc_vM1zwEsCp&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.IuwuQ-zrv75WNvlqxz67bgEsCW&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.iYQ0V8Gp-uPkQzQfs_gG3AEqDR&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.0rHZ-r0lwbzWgWBCCrUlmgEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.htbAESXGRJEb7AbLupBudAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.iYaGG8x7-tdIzpVnrYgXuAEgDY&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.4XHi9XyDm3EEb2DYXvIuPAEsDc&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.DzV84uU622idtk55B1b8bgDDEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.q4fL9aOdyq2SI05mfE-gwgEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.chgTPRZmlPXq7O41vCZpwQDIEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.S_ZYnyj1qBi8VjhYCg2_uAEsDG&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.uOGT3Y-1pmjL5DTXyHV4lgEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.3Bvwxn2XaMvB-MMWKNvFLgEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.0CtJMn-G4u08rjFVLxnrUgEsBz&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.yQPoagucXsKPrGb_Af9SuQEyDL&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.9Z-arDhJrNjsp_dLFvvooQEyDL&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.JvuyB2sDSK2taNOnV93x5gEsDG&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.yTqgE6zYCXR6-Rw42mGQqgEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.Qhn2BOcX1grSduvlOm7TxQCqDo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.-o0Ie5AZycVPAZ6xMZycTAD-Es&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.5tR4e6OqgWanRQSdyIxAzAFNC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.NUNBFsofwVDTTSSM0EEn1QC7Es&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.ca434k8pGrVxko53PCXtdQEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.GyvR0LCceYB1dd4M-xdRGQEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.Nbgl1veRKHpHO4a7cbYBJwEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.gzc97pV6bSkaLdfp0EizrAEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.93wNqRm7Y9eF-8sl0TlxSwEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.SNlmaPjEecnG2YFy7TvlaQDYEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.pr1DOxC1zwC3lOVEttrHnwEJEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.AxHw7-V6RAGucHAYVK5kKgEsDS&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.Dly3aQk9Os7881soFmUU9QEsDS&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.6tICQiqJ9kGVrpnXFCGl-wEsDd&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.6pvahpNAaqJ-M5Oe1w0o_gEsDg&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.qWG2PO3wgNQGqtuIbVySOQEsDk&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.QYVfMbHDNKZ-h0crWqmS0QEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.T_r4M5DUa3PYnmGIPTWkPAEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.qw_veNoI6dq5_0IvH2YrRQC0Bv&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP._Z8H0N1TKIGEf5HY-nxdMAEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.L7om6V-6ktaaTDdPmscJhgEsCh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.8cSrNCIIRxXf105rDzmmvADwEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.UqJ-SiAW728RkL1YhZLWvAC7FN&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.-8sppfUg-9-ZNR9Tv_XiLAEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.LVeSzBiLsilq49jCtMDonwEsCp&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.mdkJIiXz_Jrz_RLpqwZk5QEsB4&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.wg8K_fyqVaI6o2HOLhKz2QEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.OcdNQPQe65eTLhpkL0VQ_gEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.J6f93GTA63oClPzyI8Ct3AEsDW&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.mFYmUOhtZGc3eSy06Tgo9QEsDT&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.HBjUPlq8ijL7lg2AIP2R8AEsDn&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.DbKdhlymmguL-TtNVbqJcwEsC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.0zzA4uTJFHifiJWB5dJeZwEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.igMUgPUTWiazigS1ggsMIwEsDH&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse4.mm.bing.net/th?id=OIP.e-VAfpbAWBRobSxMzmP_OAEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.0i2hEuprimbuuphxwyZFYQEmDU&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.33IUPH9jhBPc7yaIae-Z3AEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.mN4H2hwDu2xE6RJbWn4bCAEsDU&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse2.mm.bing.net/th?id=OIP.5VRBhoOdxTuJ4eDqmgJ9NwFNC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.hTy7chUhJxbXRQoLEHZ_NQFNC7&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.FlvvXvVESBZkbvPaxvzYNwDZEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.8DeNgZhkUeL2VFLp4xaIWQEHEs&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.0bmV3RU_S3k__VAFgh-oKgEsDI&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.hJyJdbLcauHFE6YfRHCcjgEsCo&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.PNnGJ2oy_Ae2D3DbxgZkPwEsDM&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP._2pboFhHi7yrbGG0ehfruAEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse3.mm.bing.net/th?id=OIP.JiJL_mSEuyckBgDL7-tIiAEsDX&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300",
"https://tse1.mm.bing.net/th?id=OIP.7JxpM2x7iYVeguaUSEoKBwEsDh&amp;pid=15.1&amp;P=0&amp;w=300&amp;h=300"
]
TRAIN_IMAGE_PATHS = [ os.path.join(PATH_TO_TRAIN_IMAGES_DIR, 'basketball_{}.jpg'.format(i)) for i in range(1, len(images)+1) ]

In [ ]:
opener = urllib.request.URLopener()
for i in range(0, len(images)):
    print(images[i])
    print(TRAIN_IMAGE_PATHS[i])
    opener.retrieve(images[i], TRAIN_IMAGE_PATHS[i]) 

In [ ]:
np.random.randint(1, 141, 14)

In [ ]:
np.sort([ 27,  54, 119,  43,  10, 136, 127, 114,  60,   3,  35, 117,  84,  15])

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
for directory in ['train','test']:
    image_path = os.path.join(os.getcwd(), 'images/basketball_images/{}'.format(directory))
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('basketball_images/data/{}_labels.csv'.format(directory), index=None)
    print('Successfully converted xml to csv.')

In [6]:
# What model to download.
MODEL_NAME = 'basketball_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

NUM_CLASSES = 1

In [ ]:
PATH_TO_IMAGES_DIR = 'basketball_images'

path_train_images = os.path.join(PATH_TO_IMAGES_DIR, 'images/2_train')
path_test_images = os.path.join(PATH_TO_IMAGES_DIR, 'images/2_test')

if not os.path.exists(path_train_images):
    os.makedirs(path_train_images)
    
if not os.path.exists(path_test_images):
    os.makedirs(path_test_images)

In [ ]:
import math
np.random.seed(0)

# Images
image_paths = np.array(glob.glob(os.path.join(PATH_TO_IMAGES_DIR, "*.jpg")))
np.random.shuffle(image_paths)

train_size = math.floor(len(image_paths)*.90)

train_images = image_paths[:train_size]
test_images = image_paths[train_size:]

# Tags
train_images_tags = [filename[:-4] + '.xml' for filename in train_images]
test_images_tags = [filename[:-4] + '.xml' for filename in test_images]


In [ ]:
test_images_tags

In [ ]:
for file_path in train_images:
    if os.path.exists(file_path):
        os.rename(file_path, os.path.join(path_train_images, os.path.basename(file_path)))
    
for file_path in test_images:
    if os.path.exists(file_path):
        os.rename(file_path, os.path.join(path_test_images, os.path.basename(file_path)))
 
for file_path in train_images_tags:
    if os.path.exists(file_path):
        os.rename(file_path, os.path.join(path_train_images, os.path.basename(file_path)))
    
for file_path in test_images_tags:
    if os.path.exists(file_path):
        os.rename(file_path, os.path.join(path_test_images, os.path.basename(file_path)))

In [ ]:
test_images

In [ ]:
for directory in ['train','test']:
    image_path = os.path.join(os.getcwd(), 'basketball_images/images/{}'.format(directory))
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('basketball_images/data/{}_labels.csv'.format(directory), index=None)
    print('Successfully converted xml to csv.')

In [ ]:
#python generate_tfrecord.py --csv_input=object_detection/basketball_images/data/train_labels.csv --output_path=object_detection/basketball_images/data/train.record --image_path=object_detection/basketball_images/images/train
#python generate_tfrecord.py --csv_input=object_detection/basketball_images/data/test_labels.csv --output_path=object_detection/basketball_images/data/test.record --image_path=object_detection/basketball_images/images/test

#python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-183 --output_directory basketball_inference_graph